In [1]:
import pandas as pd
# Imports
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
from src.data.make_dataset import CustomDataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset
import src.models.featureExtractor as Model
from torchvision import transforms




In [2]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [3]:
MODEL_PATH = '../models'

In [4]:
dataset = CustomDataset(csv_file='../data/interim/train/train.csv', transform=transforms.ToTensor())

train_set, test_set = random_split(dataset, [.8, .2])
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)
#
# videos, labels = next(iter(train_loader))
# print(labels)

In [5]:
# videos, labels = next(iter(train_loader))
# print(videos.shape)

In [6]:
# BATCH_SIZE = 64
NUM_EPOCHS = 200

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# Initialize the lstm model
model = Model.model
model.to(device)
# Initialize loss and optimizer
criterion = nn.BCELoss()
optimizer = Model.optimizer

cuda


In [8]:
def save_model(best_model, best_optimizer, rmse_test, n_epoch):
    state = {
                'epoch': n_epoch,
                'model_state_dict': best_model.state_dict(),
                'optimizer_state_dict': best_optimizer.state_dict(),
                'loss': rmse_test,
                }
    torch.save(state, MODEL_PATH + f'/570_alternative_approach_{n_epoch}_{round(float(rmse_test), 2 )}.tar')

In [9]:
# check the accuracy
def check_acc(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        print('here...................')
        for x,y in loader:
            x = x.type(torch.FloatTensor).to(device=device)
            y = y.type(torch.FloatTensor).to(device=device).unsqueeze(1)

            # Calculate the acc
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = num_correct/ num_samples

    model.train()
    return acc
        # print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct/ num_samples) * 100:.02f} ')


In [ ]:
# Train Networks
for epoch in range(NUM_EPOCHS):
    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        # Get data cuda if possible
        data = data.type(torch.FloatTensor).to(device=device)
        targets = targets.type(torch.FloatTensor).to(device=device).unsqueeze(1)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        # Gradient step or Adam step
        optimizer.step()
    acc_train = check_acc(train_loader, model)
    acc_test  = check_acc(test_loader, model)
    print('Train Epoch: {} \ttrain_Loss: {:.02f} \ttest_Loss: {:.02f}'.format(epoch,acc_train * 100, acc_test * 100))


4it [08:36, 127.45s/it]